# Query Blockchain Data
**Purpose:** Learn how to easily query raw blockchain data via APIs from leading blockchain analytics platforms.

Free:
* [Flipside](https://flipsidecrypto.xyz/)
* [Dune](https://dune.com/home)
* [BigQuery](https://console.cloud.google.com/marketplace/browse)
* [Transpose](https://www.transpose.io/)
* [Chainbase](https://chainbase.online/)

Paid:
* [Spice](https://spice.xyz/)
* [Footprint](https://footprint.network/)

Snowflake:
* [Coherent](https://coherent.xyz/)
* [Token Flow](https://tokenflow.live/)
* [Credmark](https://credmark.com/)


In [ ]:
import json
import pandas as pd
import os
import json
from shroomdk import ShroomDK
from google.cloud import bigquery, bigquery_storage, storage
import time
import requests
import math
from spicepy import Client
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL
from transpose import Transpose
import datetime

In [ ]:
with open("keys.json", "r") as keys_file:
    keys = json.load(keys_file)
    flipside_key = keys["flipside_key"]
    chainbase_key = keys["chainbase_key"]
    makerdojo_key = keys["makerdojo_key"]
    spice_key = keys["spice_key"]
    footprint_key = keys["footprint_key"]
    transpose_key = keys["transpose_key"]
    snowflake_account = keys["snowflake_account"]
    snowflake_user = keys["snowflake_user"]
    snowflake_password = keys["snowflake_password"]
    snowflake_warehouse = keys["snowflake_warehouse"]
    dune_key = keys["dune_key"]

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "keys.json"

In [ ]:
def run_query(q, provider):
    start = datetime.datetime.now()

    # Dictionary of provider names and their respective query functions
    provider_query = {
        "flipside": query_flipside,
        "transpose": query_transpose,
        "bigquery": query_bigquery,
        "dune": query_dune,
        "chainbase": query_chainbase,
        "spice": query_spice,
        "coherent": query_snowflake,
        "tokenflow": query_snowflake,
        "credmark": query_snowflake,
        "footprint": query_footprint,
    }
    df = provider_query[provider](q)

    end = datetime.datetime.now()
    runtime = int((end - start).total_seconds())
    df["runtime_sec"] = runtime
    df["provider"] = provider

    return df

## [Flipside](https://flipsidecrypto.xyz/)

In [ ]:
# Query Flipside using their Python SDK
def query_flipside(q):
    sdk = ShroomDK(flipside_key)
    result_list = []
    for i in range(1, 11):  # max is a million rows @ 100k per page
        data = sdk.query(q, page_size=100000, page_number=i)
        if data.run_stats.record_count == 0:
            break
        else:
            result_list.append(data.records)
    result_df = pd.DataFrame()
    for idx, each_list in enumerate(result_list):
        if idx == 0:
            result_df = pd.json_normalize(each_list)
        else:
            try:
                result_df = pd.concat([result_df, pd.json_normalize(each_list)])
            except:
                continue
    result_df.drop(columns=["__row_index"], inplace=True)
    return result_df

In [ ]:
q = """
select 
    min(block_number) first_block_num
    , min(block_timestamp) first_block_dttm
    , max(block_number) last_block_num
    , max(block_timestamp) last_block_dttm
    , datediff('second', max(block_timestamp), current_timestamp) latency_sec
    , count(1) total_blocks
from ethereum.core.fact_blocks
"""
flipside_df = run_query(q, "flipside")
flipside_df

## [Dune](https://dune.com/home)

In [ ]:
# Query Dune Analytics using API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/2296642/results?api_key={dune_key}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [ ]:
# https://dune.com/queries/2296642
q = "2296642"
dune_df = run_query(q, "dune")
dune_df

## [Google BigQuery](https://console.cloud.google.com/marketplace/browse)

In [ ]:
# Query BigQuery using their Python SDK
def query_bigquery(q):
    connection = bigquery_storage.BigQueryReadClient()
    bqclient = bigquery.Client()
    results_df = bqclient.query(q).result().to_dataframe(bqstorage_client=connection)
    return results_df

In [ ]:
q = """
select
    min(number) as first_block_num
    , min(`timestamp`) as first_block_dttm 
    , max(number) as last_block_num
    , max(`timestamp`) as last_block_dttm
    , timestamp_diff(current_timestamp(), max(`timestamp`), second) latency_sec
    , count(1) total_blocks
from `bigquery-public-data.crypto_ethereum.blocks`
"""
bigquery_df = run_query(q, "bigquery")
bigquery_df

## [Transpose](https://www.transpose.io/)

In [ ]:
# Query Transpose using their API
def query_transpose(q):
    url = "https://api.transpose.io/sql"
    headers = {
        "Content-Type": "application/json",
        "X-API-KEY": transpose_key,
    }
    response = requests.post(
        url,
        headers=headers,
        json={
            "sql": q,
        },
    )
    results_json = json.loads(response.text)["results"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [ ]:
q = f"""
select 
    min(block_number) first_block_num
    , min(timestamp) first_block_dttm
    , max(block_number) last_block_num
    , max(timestamp) last_block_dttm
    , EXTRACT(EPOCH FROM NOW() - MAX(blocks.timestamp))::INTEGER latency_sec
    , count(1) total_blocks
from ethereum.blocks
"""
transpose_df = run_query(q, "transpose")
transpose_df

## [Chainbase](https://chainbase.online/)

In [ ]:
# Query Chainbase using their REST API
def query_chainbase(q):
    url = "https://api.chainbase.online/v1/dw/query"
    payload = {"query": q}
    headers = {"x-api-key": chainbase_key}
    response = requests.post(url, json=payload, headers=headers)
    task_id = response.json()["data"]["task_id"]
    row_ct = response.json()["data"]["rows"]
    all_results_df = pd.DataFrame([])
    row_ct = math.ceil(response.json()["data"]["rows"] / 1000)
    if row_ct == 1:
        data = response.json()["data"]["result"]
        all_results_df = pd.json_normalize(data)
    else:
        time.sleep(2)
        for i in range(0, row_ct, 1):
            payload = {"task_id": task_id, "page": i + 1}
            response = requests.post(url, json=payload, headers=headers)
            data = response.json()["data"]["result"]
            results_df = pd.json_normalize(data)
            all_results_df = pd.concat([all_results_df, results_df])
            time.sleep(2)
    return all_results_df

In [ ]:
q = """
select 
    min(number) first_block_num
    , min(timestamp) first_block_dttm
    , max(number) last_block_num
    , max(timestamp) last_block_dttm
    , (now() - max(timestamp)) latency_sec
    , count(1) total_blocks
from ethereum.blocks
"""
chainbase_df = run_query(q, "chainbase")
chainbase_df

## [Spice](https://spice.xyz/)

In [ ]:
# Query Spice using their SDK
def query_spice(q):
    client = Client(spice_key)
    reader = client.query(q)
    results_df = reader.read_pandas()
    return results_df

In [ ]:
q = """
select 
    min(number) first_block_num
    , to_timestamp(min("timestamp")) first_block_dttm
    , max(number) last_block_num
    , to_timestamp(max("timestamp")) last_block_dttm
    , timestampdiff(second, to_timestamp(max("timestamp")), current_timestamp) latency_sec
    , count(1) total_blocks
from eth.blocks
"""
spice_df = run_query(q, "spice")
spice_df

## [Coherent](https://coherent.xyz/)

In [ ]:
# Query Coherent using their Snowflake API
def query_snowflake(q):
    conn = create_engine(
        URL(
            account=snowflake_account,
            user=snowflake_user,
            password=snowflake_password,
            warehouse=snowflake_warehouse,
        )
    )
    connection = conn.raw_connection()
    q_results = connection.cursor().execute(q)
    result_df = q_results.fetch_pandas_all()
    result_df.columns = result_df.columns.str.lower()
    return result_df

In [ ]:
q = """
"""
# coherent_df = run_query(q, "coherent")
# coherent_df

## [Credmark](https://credmark.com/)

In [ ]:
q = """
"""
# credmark_df = run_query(q, "credmark")
# credmark_df

## [Token Flow](https://tokenflow.live/)

In [ ]:
q = """
select 
    min(block) first_block_num
    , min(timestamp) first_block_dttm
    , max(block) last_block_num
    , max(timestamp) last_block_dttm
    , datediff('second', max(timestamp), current_timestamp) latency_sec
    , count(1) total_blocks
from ethereum_data_warehouse_beta.raw.blocks
"""
tokenflow_df = run_query(q, "tokenflow")
tokenflow_df

## [Footprint](https://footprint.network/)

In [ ]:
# Query Footprint using their API
def query_footprint(q):
    url = "https://api.footprint.network/api/v1/native"

    payload = {"query": q}
    headers = {
        "accept": "application/json",
        "API-KEY": footprint_key,
        "content-type": "application/json",
    }

    response = requests.post(url, json=payload, headers=headers)
    results_json = json.loads(response.text)["data"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [ ]:
q = """
select 
    max(number) latest_block_num
    , max(timestamp) latest_block_dttm
    , date_diff('second', max(timestamp), now()) latency_seconds
from ethereum_blocks
"""
footprint_df = run_query(q, "footprint")
footprint_df

## Compare Results

In [ ]:
combined_df = pd.concat(
    [
        flipside_df,
        transpose_df,
        bigquery_df,
        dune_df,
        chainbase_df,
        spice_df,
        tokenflow_df,
    ]
)
combined_df = (
    combined_df.sort_values(  # [["provider", "total_blocks", "latency_sec", "runtime_sec"]]
        by="latency_sec", ascending=True
    )
    .reset_index(drop=True)
    .assign(total_blocks=lambda x: x["total_blocks"].astype(int))
    .set_index("provider")
)
heatmap = combined_df.style.background_gradient(cmap="Blues")
heatmap